# Introduction à Langchain 



In [1]:
%pip install langchain langchain-openai langchain_mistralai openai python-dotenv -q

Note: you may need to restart the kernel to use updated packages.


### Le modèle de LLM

Basiquement Langchain nécessite simplement d'instancier un `Runnable` pour le modèle de LLM et de l'invoquer directement

Langchain fournit une intégration pour un grand nombre de [providers et de modèles](https://python.langchain.com/v0.2/docs/integrations/chat/). Elles partagent une interface commune, permettant de switcher facilement. Attention, tous les modèles ne supportent pas les mêmes fonctionnalités (streaming, structure json, usage de token, etc.)

<img src="./assets/runnable_model.png" alt="drawing" width="400"/>

In [3]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from os import getenv
load_dotenv()

llm_openai = ChatOpenAI(model="gpt-4o-mini", api_key=getenv('OPENAI_API_KEY'))

On peut invoquer le runnable (et donc le model directement).
C'est une abstraction au dessus de :

```python 
client = OpenAI(api_key=getenv('OPENAI_API_KEY'))
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explique moi la théorie de la relativité en termes simples",
        }
    ],
    model="gpt-4o-mini"
)

```

In [4]:
response_4o_mini = llm_openai.invoke("Hello, how are you today?")
response_4o_mini

AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 14, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-d29a22f8-dad8-4ff4-b8dc-eb3697d48ca4-0', usage_metadata={'input_tokens': 14, 'output_tokens': 31, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

La complétion consiste en un objet `AIMessage`
* **content** - la complétion
* **response_metadata** - Metadata additionnels. Par exemple: response headers, logprobs, token counts.

In [19]:
print(response_4o_mini.content)
print(response_4o_mini.response_metadata)

Hello! I'm just a computer program, but I'm ready and here to help you. How can I assist you today?
{'token_usage': {'completion_tokens': 24, 'prompt_tokens': 14, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}


Pour appeler un autre modèle, par exemple un de Mistral on switch simplement le modèle créé. Tout reste identique.

In [5]:

from langchain_mistralai import ChatMistralAI
llm_mistral = ChatMistralAI(model="mistral-large-latest", api_key=getenv('MISTRAL_API_KEY'))
response_mistral = llm_mistral.invoke("Hello, how are you today?")

response_mistral

AIMessage(content="Hello! I'm functioning as intended, thank you. How about you? How's your day going?", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 10, 'total_tokens': 33, 'completion_tokens': 23}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-75ce1669-78e8-4283-ac74-19de825df17b-0', usage_metadata={'input_tokens': 10, 'output_tokens': 23, 'total_tokens': 33})

In [6]:
print(response_mistral.content)
print(response_mistral.response_metadata)

Hello! I'm functioning as intended, thank you. How about you? How's your day going?
{'token_usage': {'prompt_tokens': 10, 'total_tokens': 33, 'completion_tokens': 23}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}


### Le prompt

Nous avons invoqué le modèle en lui passant directement le prompt comme `string`. Très souvent, et encore plus lorsque l'on voudra *augmenter* le modèle, notre prompt n'est pas une chaîne en dure mais composée de pluseirus partie, certaines potentiellement variables.

Typiquement le prompt se décompose de deux parties
* La partie `system` ou `preprompt`, qui peut par exemple contenir des instructions pour le modèle (par exemple de formattage de la réponse),  
* La partie `user` contenant la requête de l'utilisateur à proprement parlé.